# Jan Izydorczyk
# Sprawozdanie 7

In [2]:
from matplotlib import pyplot as plt
import math
import numpy as np
import random
import scipy
import sys
%matplotlib notebook

print("Wersja Pythona:")
print (sys.version)
print(f"Wersja numpy: {np.__version__}")
print("Konfiguracja liczb zmiennoprzecinkowych")
print(sys.float_info)

f64 = np.float64
f32 = np.float32

Wersja Pythona:
3.9.4 (tags/v3.9.4:1f2e308, Apr  6 2021, 13:40:21) [MSC v.1928 64 bit (AMD64)]
Wersja numpy: 1.20.2
Konfiguracja liczb zmiennoprzecinkowych
sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)


In [73]:
from scipy.linalg import lu_factor, lu_solve

# Zadanie 1

In [ ]:
def eig_(method):
    def wrapper(A, b, sol=None, epsilon=None, *args, x_t=None, max_steps=None, epsilons_buf=None,  **kwargs):
        A, b = remove_zeros_from_diagonal(A, b)
        maxes = np.column_stack((np.max(np.abs(A), axis=1),b))
        maxes = np.max(maxes, axis=1).reshape(-1, 1)
        A /= maxes
        b /= maxes
        if x_t is None:
            x_t = np.random.rand(*b.shape)
        else:
            x_t = np.copy(x_t)
        def pred(x,steps):
            nonlocal epsilon, sol
            res = False
            if sol is not None and epsilon is not None:
                res =  not np.allclose(x, sol, atol=epsilon)
            elif max_steps is not None:
                res = steps < max_steps
            if res and sol is not None and epsilons_buf is not None:
                epsilons_buf.append(np.mean(np.abs(x - sol)))
            return res
        return method(A, b, sol, epsilon, *args, x_t=x_t, max_steps=max_steps, epsilons_buf=epsilons_buf, pred=pred, **kwargs)
    return wrapperdef solver_(method):
    def wrapper(A, b, sol=None, epsilon=None, *args, x_t=None, max_steps=None, epsilons_buf=None,  **kwargs):
        A, b = remove_zeros_from_diagonal(A, b)
        maxes = np.column_stack((np.max(np.abs(A), axis=1),b))
        maxes = np.max(maxes, axis=1).reshape(-1, 1)
        A /= maxes
        b /= maxes
        if x_t is None:
            x_t = np.random.rand(*b.shape)
        else:
            x_t = np.copy(x_t)
        def pred(x,steps):
            nonlocal epsilon, sol
            res = False
            if sol is not None and epsilon is not None:
                res =  not np.allclose(x, sol, atol=epsilon)
            elif max_steps is not None:
                res = steps < max_steps
            if res and sol is not None and epsilons_buf is not None:
                epsilons_buf.append(np.mean(np.abs(x - sol)))
            return res
        return method(A, b, sol, epsilon, *args, x_t=x_t, max_steps=max_steps, epsilons_buf=epsilons_buf, pred=pred, **kwargs)
    return wrapper

## Metoda potęgowa

In [105]:
def eig_(method):
    def wrapper(M, *args, x_0=None, epsilon=None, max_steps=None, **kwargs):
        
        def pred(x_t1, x_t, steps):
            nonlocal max_steps, epsilon
            if epsilon is not None:
                if steps > 2e4:
                    print('Overflow')
                    print('----')
                    print(np.linalg.norm(x_t1 - x_t))
                    return False
                return np.linalg.norm(x_t1 - x_t) > epsilon and np.linalg.norm(x_t1 + x_t) > epsilon
            elif max_steps is not None:
                return steps < max_steps
            else:
                return False
            
        if x_0 is None:
            x_0 = np.random.rand(M.shape[0], 1)
        else:
            x_0 = np.copy(x_0)
            
        return method(M, *args, x_0=x_0, epsilon=epsilon, max_steps=max_steps, pred=pred, **kwargs)
    return wrapper

In [106]:
@eig_
def power_method(M, x_0=None, epsilon=None, max_steps=None, pred=None):
    x_t = x_0
    steps = 0
    x = x_t + 2
    
    while pred(x_t, x, steps):
        steps += 1
        x = x_t
        x_t = M @ x_t
        idx = np.argmax(np.abs(x_t))
        max_val = x_t[idx, 0]
        x_t /= max_val
    return max_val, x_t / np.linalg.norm(x_t)
        

In [107]:
def rand_symm(N):
    M = np.random.rand(N, N)
    return (M + M.T) / 2

In [108]:
M = rand_symm(4)
print(M)

own_eig, own_qig_vec = power_method(M, epsilon=1e-5)
np_eig, np_eig_vec = np.linalg.eig(M)
np_eig, np_eig_vec = np_eig[0], np_eig_vec[:, 0]

print(own_eig)
print(own_qig_vec.flatten())


print()
print(np_eig)
print(np_eig_vec.flatten())


[[0.75545675 0.28732318 0.62540451 0.63543743]
 [0.28732318 0.4473667  0.37764237 0.27239405]
 [0.62540451 0.37764237 0.56999861 0.51449393]
 [0.63543743 0.27239405 0.51449393 0.17733435]]
1.9224998588377684
[0.61989084 0.3431946  0.55043845 0.44155443]

1.922492554708524
[0.61989082 0.34319428 0.55043834 0.44155486]


In [109]:
@eig_
def inv_power_method(M, sigma, x_0=None, epsilon=None, max_steps=None, pred=None):
    x_t = x_0
    steps = 0
    x = x_t + 2
    
    M = (M - np.eye(M.shape[0]) * sigma)
    lu, piv = lu_factor(M)
    while pred(x_t, x, steps):
        steps += 1
        x = np.copy(x_t)
        x_t = lu_solve((lu, piv), x_t)
        max_val = np.linalg.norm(x_t, ord=np.inf)
        x_t /= max_val
    return 1 / max_val + sigma, x_t / np.linalg.norm(x_t)

In [110]:
M = rand_symm(4)
print(M)
print()

np_eig, np_eig_vec = np.linalg.eig(M)
np_eig, np_eig_vec = np_eig[0], np_eig_vec[:, 0]

own_eig, own_qig_vec = inv_power_method(M, sigma=np_eig + 1e-7, epsilon=1e-3)

print(own_eig)
print(own_qig_vec.flatten())


print()
print(np_eig)
print(np_eig_vec.flatten())

[[0.75517109 0.6582062  0.43968271 0.25315517]
 [0.6582062  0.22443669 0.06092191 0.63891043]
 [0.43968271 0.06092191 0.12702539 0.27221548]
 [0.25315517 0.63891043 0.27221548 0.19305966]]

1.6129436175404515
[0.67938621 0.52619231 0.29865991 0.41516056]

1.6129434175404493
[0.67938621 0.52619231 0.29865991 0.41516056]


In [ ]:
@eig_
def inv_power_method_with_rayleigh(M, sigma, x_0=None, epsilon=None, max_steps=None, pred=None):
    x_t = x_0
    steps = 0
    x = x_t + 2
    
    M = (M - np.eye(M.shape[0]) * sigma)
    lu, piv = lu_factor(M)
    while pred(x_t, x, steps):
        steps += 1
        x = np.copy(x_t)
        x_t = lu_solve((lu, piv), x_t)
        max_val = np.linalg.norm(x_t, ord=np.inf)
        x_t /= max_val
    return 1 / max_val + sigma, x_t / np.linalg.norm(x_t)